In [1]:
# Notebook phân tích & xây dựng mapping cho dữ liệu bất động sản
import os
import sys
from datetime import datetime
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

from pyspark.sql import SparkSession, DataFrame
from pyspark.sql.functions import (
    col, to_timestamp, current_timestamp, lit, regexp_replace, trim,
    when, upper, lower, split, element_at, round as spark_round,
    avg, count, percentile_approx, stddev, min as spark_min, max as spark_max,
    udf, length, expr
)
from pyspark.sql.types import StringType, DoubleType, BooleanType
from pyspark.sql.window import Window

# Thêm thư mục gốc vào sys.path
project_root = os.path.abspath(os.path.join(os.getcwd(), "../.."))
sys.path.append(project_root)

# Tạo Spark Session
spark = SparkSession.builder \
    .appName("BatDongSan Mapping Analysis") \
    .config("spark.ui.port", "4050") \
    .config("spark.driver.memory", "4g") \
    .config("spark.executor.memory", "4g") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://namenode:9000") \
    .getOrCreate()

print("Spark session created successfully")

Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/06/22 17:50:27 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


Spark session created successfully


In [10]:
parquet_file = "/data/realestate/raw/batdongsan/house/2025/05/25/*.json"
df = spark.read.json(parquet_file)

# Lọc các bản ghi có location == "sfs"
filtered_df = df.filter(df["title"] == "MẶT TIỀN KINH DOANH-KHU VIP TÂN PHÚ - GẦN NGUYỄN SƠN - 4x 15.2 - 3 TẦNG- VỊ TRÍ AN NINH - CHỈ 6.99T")

# Hiển thị kết quả (tuỳ chọn)
filtered_df.select("source", "location", "url").show(100)

+----------+--------------------+--------------------+
|    source|            location|                 url|
+----------+--------------------+--------------------+
|batdongsan|MẶT TIỀN KINH DOA...|https://batdongsa...|
+----------+--------------------+--------------------+



In [13]:
# from pyspark.sql.functions import col

# parquet_file = "/data/realestate/raw/batdongsan/house/2025/05/25/*.json"
# df = spark.read.json(parquet_file)

# # Xoá bản ghi theo điều kiện
# df_cleaned = df.filter(col("title") != "MẶT TIỀN KINH DOANH-KHU VIP TÂN PHÚ - GẦN NGUYỄN SƠN - 4x 15.2 - 3 TẦNG- VỊ TRÍ AN NINH - CHỈ 6.99T")

# # Ghi đè lên chính thư mục gốc (CẨN THẬN: sẽ xoá toàn bộ file cũ)
df_cleaned.write.mode("overwrite").json("/data/realestate/raw/batdongsan/house/2025/05/25/")


25/06/22 18:02:19 ERROR Utils: Aborting task
org.apache.spark.SparkFileNotFoundException: File does not exist: /data/realestate/raw/batdongsan/house/2025/05/25/property_data_2025_05_25_134716_134322.json
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:86)
	at org.apache.hadoop.hdfs.server.namenode.INodeFile.valueOf(INodeFile.java:76)
	at org.apache.hadoop.hdfs.server.namenode.FSDirStatAndListingOp.getBlockLocations(FSDirStatAndListingOp.java:158)
	at org.apache.hadoop.hdfs.server.namenode.FSNamesystem.getBlockLocations(FSNamesystem.java:1954)
	at org.apache.hadoop.hdfs.server.namenode.NameNodeRpcServer.getBlockLocations(NameNodeRpcServer.java:755)
	at org.apache.hadoop.hdfs.protocolPB.ClientNamenodeProtocolServerSideTranslatorPB.getBlockLocations(ClientNamenodeProtocolServerSideTranslatorPB.java:439)
	at org.apache.hadoop.hdfs.protocol.proto.ClientNamenodeProtocolProtos$ClientNamenodeProtocol$2.callBlockingMethod(ClientNamenodeProtocolProtos.java)
	at org.ap

Py4JJavaError: An error occurred while calling o153.json.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 20.0 failed 1 times, most recent failure: Lost task 0.0 in stage 20.0 (TID 20) (7dd3ccea0838 executor driver): org.apache.spark.SparkException: [TASK_WRITE_FAILED] Task failed while writing rows to hdfs://namenode:9000/data/realestate/raw/batdongsan/house/2025/05/25.
	at org.apache.spark.sql.errors.QueryExecutionErrors$.taskFailedWhileWritingRowsError(QueryExecutionErrors.scala:788)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.executeTask(FileFormatWriter.scala:420)
	at org.apache.spark.sql.execution.datasources.WriteFilesExec.$anonfun$doExecuteWrite$1(WriteFiles.scala:100)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2(RDD.scala:888)
	at org.apache.spark.rdd.RDD.$anonfun$mapPartitionsInternal$2$adapted(RDD.scala:888)
	at org.apache.spark.rdd.MapPartitionsRDD.compute(MapPartitionsRDD.scala:52)
	at org.apache.sp